In [ ]:
import pandas as pd
import numpy as np
from itertools import accumulate
import os

# Format Data for Neo4j Import

In this notebook we format data to load into Neo4j.  Our source will be the training dataset used in the [CIKM Cross-Device Entity Linking Challenge](https://competitions.codalab.org/competitions/11171). The raw dataset can be found [here](https://drive.google.com/file/d/0B7XZSACQf0Kdc3BnZzdEZzR6X0k/view?usp=sharing&resourcekey=0-KcizLV8DrV0jw_VLPNLvxQ).  

__To run this notebook please do the following:__
 - create a subdirectory called `./data`
 - download the `data-train-dca.zip` file from [here](https://drive.google.com/drive/folders/0B7XZSACQf0KdNXVIUXEyVGlBZnc?resourcekey=0-7ITozWjtDNvBHfTROIfxLg) (~850MB) and unzip into `./data` directory. 
 
__Note: Depending on how you set the parameters below, this notebook could take 10 or more minutes to run completely__

### Summary 
This notebook subsamples the raw data based on the `SAMPLE_RATE` defined below and transforms it into five input data sets

1. __(Users)__: Nodes Representing Users


2. __(Users)-\[SAME_AS\]->(Users)__: Relationships representing labeled Entity-Linkage/Alignments between Users.  Will be used for supervised link prediction. 


3. __(Websites)__: Nodes representing websites. Each URL path from the input dataset will be decomposed into multiple websites to represent the path hierarchy. The depth to consider for the hierarchy is set by the `MAX_URL_SEGMENTS` variable below. Each URL will be split into `MAX_URL_SEGMENTS + 1` website nodes where `MAX_URL_SEGMENTS` website nodes will be used for the hierarchy and the last website node will correspond to the the full url path.  For example, if `MAX_URL_SEGMENTS=3`, then the URL `a/b/c/d/e` will become 4 website nodes identified by the urls `[a, a/b, a/b/c, a/b/c/d/e]` while the website `aa/bb/cc` will become `[aa, aa/bb, aa/bb/cc, aa/bb/cc]`. The website nodes will be deduplicated so each unique url in the hierarchy is only represented by a singe node in the graph. This is done because the full url paths are fairly unique and provide only sparse connectivity between users.  Splitting urls in this way allows us to draw meaningful relationships from which we can engineer features for link prediction. 


4. __(User)-\[VISITED\]->(Website)__: Relationships representing users interacting with websites. The relationships will be weighted by number of visits.


5. __(Website)-\[CHILD_OF\]->(Website)__: Relationships representing hierarchical structure of websites. i.e. if the URL `a/b/c/d` is split into `[a, a/b, a/b/c, a/b/c/d]` per above rules, then there would be a `CHILD_OF` relationship going from `a/b/c/d -> a/b/c`, `a/b/c -> a/b`, and `a/b -> a`.  This allows for a much more well connected graph. 

## Parameters for Sampling and Transformation

In [ ]:
# The rate between(0,1) to sample User Ids from facts.json. 
# Recommend using <=0.1 for quick experimentation
SAMPLE_RATE = 0.1

# Whether to sample only pairs of nodes that are aligned in the training set. 
# This will bias the graph to a higher true class ratio if set to True
ONLY_SAMPLE_ALIGNED_PAIRS = False

# The number of URL segments to consider for the URLs
MAX_URL_SEGMENTS = 5

# The Random Seed to use throughout
RANDOM_SEED = 7474

## Format Facts (a.k.a Logged Events)

We load facts.json to sample from it and shape a unique __userid->eventid__ data frame.
The size of the data set may limit the ability to load it all at once depending on your environment so the below example iterates over the json file in chunks.  Feel free to tune the paramters so it is optimized for your machine. 

In [ ]:
#helper functions
def extract_values_from_facts(row):
    return row.facts['fid'], row.facts['ts']

def format_facts(raw_df):
    df = raw_df.explode("facts")
    df[['fid','ts']] = df.apply(lambda row: extract_values_from_facts(row), axis=1, result_type ='expand')
    #df.drop(columns = ["facts"], inplace=True)
    return df

In [ ]:
# Labeled Entity-Linkages/Alignments
raw_train_df = pd.read_csv('./data/data-train-dca/train.csv', header=0, names=['uid1', 'uid2'])

In [ ]:

#iterative sampling of facts.json
n = 0
facts_dfs = []
chunk_size = 5000
if ONLY_SAMPLE_ALIGNED_PAIRS:
    sample_train_df = raw_train_df.sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
    unique_aligned_uids = pd.concat([sample_train_df.uid1, sample_train_df.uid2], ignore_index=True) \
    .drop_duplicates().tolist()
    with pd.read_json('./data/data-train-dca/facts.json', lines=True, chunksize=chunk_size) as reader:
        for chunk_df in reader:
            is_aligned_indicator = chunk_df.uid.isin(unique_aligned_uids)
            # join on alignments present, sub-sample, and format
            facts_dfs.append(format_facts(chunk_df[is_aligned_indicator]))
            #print progress
            n = n + chunk_size
            # print(n)
else:
    unique_aligned_uids = pd.concat([raw_train_df.uid1, raw_train_df.uid2], ignore_index=True) \
    .drop_duplicates().tolist()
    with pd.read_json('./data/data-train-dca/facts.json', lines=True, chunksize=chunk_size) as reader:
        for chunk_df in reader:
            is_aligned_indicator = chunk_df.uid.isin(unique_aligned_uids)
            # join on alignments present, sub-sample, and format
            chunk_df_aligned = chunk_df[is_aligned_indicator].sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
            facts_dfs.append(format_facts(chunk_df_aligned ))
            # anti-join on alignments present
            chunk_df_not_aligned = chunk_df[~is_aligned_indicator].sample(frac=SAMPLE_RATE, random_state = RANDOM_SEED)
            facts_dfs.append(format_facts(chunk_df_not_aligned ))
            #print progress
            n = n + chunk_size
            # print(n)

In [ ]:
facts_df = pd.concat(facts_dfs, ignore_index=True)
facts_df.shape

In [ ]:
clean_facts_df = facts_df.drop_duplicates(subset = ["uid", "fid", "ts"])
clean_facts_df

## Subsample Training Alignments based on Facts Subsampling

In [ ]:
unique_uid_df = clean_facts_df.uid.drop_duplicates().to_frame(name="uid")
## merge on both alignment columns
alignment_df = raw_train_df.merge(unique_uid_df, left_on="uid1", right_on="uid") \
.merge(unique_uid_df, left_on="uid2", right_on="uid")

## Join URLS to Capture Site Visits

In [ ]:
urls_df = pd.read_csv('./data/data-train-dca/urls.csv', header=0, names=['fid', 'url'])
# urls_df

In [ ]:
# Note that neither fid nor url have duplicate values
print(urls_df.fid.duplicated().sum())
print(urls_df.url.duplicated().sum())

In [ ]:
# merge facts and url visits on fid.
user_web_df = clean_facts_df.merge(urls_df, on="fid")

In [ ]:
# split the urls out by path segments so there are seperate columns for the path hierarchy
def agg_path(s1, s2):
    return s1 + '/' + s2

def url_parse(url):
    query_pos = url.find('?')
    if query_pos != -1:
        url = url[0:query_pos]
    url_path = url.split('/')
    
    url_path_length = len(url_path)

    if url_path_length>=MAX_URL_SEGMENTS:
        url_agg_paths = list(accumulate(url_path[0:MAX_URL_SEGMENTS], agg_path))
    else:
        url_agg_paths_staging = list(accumulate(url_path[0:url_path_length], agg_path))
        url_agg_paths = url_agg_paths_staging + [url_agg_paths_staging[-1]] * (MAX_URL_SEGMENTS - url_path_length)
        
    return  url_agg_paths + [url_path_length]   

url_segment_names = ["url_segment_" + str(x) for x in np.arange(0, MAX_URL_SEGMENTS, 1)]
url_new_column_names = url_segment_names + ["url_path_depth"]

In [ ]:
user_web_df[url_new_column_names] = \
    user_web_df.apply(lambda row: url_parse(row.url), axis=1, result_type ='expand')
#user_web_df

## Format and Write CSVs

In [ ]:
output_path = './data/clean'
if not os.path.exists(output_path):
    os.mkdir(output_path)
    print(f'Created new directory: {output_path}')
else:
    print(f'Directory {output_path} already exists')

In [ ]:
## Users
user_web_df['uid'].drop_duplicates().to_csv(f'{output_path}/users.csv', index=False)

In [ ]:
## (User)-[SAME_AS]->(User)
alignment_df.to_csv(f'{output_path}/user_alignments.csv', index=False)

In [ ]:
## (Website)-[:CHILD_OF]->(Website)

all_url_column_names = url_segment_names + ['url']

web_hierarchy_dfs = []

for i in range(len(all_url_column_names)-1):
    url1 = all_url_column_names[i]
    url2 = all_url_column_names[i + 1]

    web_hier_segment_df = user_web_df[[url1, url2]]
    web_hier_segment_df = web_hier_segment_df.rename(columns={url1:"url1", url2:"url2"})
    web_hier_segment_df = web_hier_segment_df[web_hier_segment_df.url1 != web_hier_segment_df.url2]
    web_hierarchy_dfs.append(web_hier_segment_df)

web_hierarchy_df = pd.concat(web_hierarchy_dfs, ignore_index=True).drop_duplicates()

web_hierarchy_df.to_csv(f'{output_path}/web_hierarchy.csv', index=False)
#web_hierarchy_df

In [ ]:
# Websites
websites_dfs = []
for url in all_url_column_names:  
    websites_df_staging = user_web_df[[url, "url_path_depth"]]
    if url != 'url':
        websites_df_staging = websites_df_staging.rename(columns={url:"url"})
        websites_df_staging = websites_df_staging.drop(columns="url_path_depth")
        websites_df_staging = websites_df_staging.assign(url_path_depth=int(url[-1]) + 1)
    websites_dfs.append(websites_df_staging)  
    
websites_df = pd.concat(websites_dfs, ignore_index=True).drop_duplicates(subset="url")
websites_df.to_csv(f'{output_path}/websites.csv', index=False)
#websites_df

In [ ]:
## (User)-[:VISITED]->(Website)
def sorted_list(x):
    res = x.tolist()
    res.sort()
    return res

users_web_visits_df = user_web_df[['uid','url','fid','ts']]
users_web_visits_df = user_web_df[['uid','url','facts','fid','ts']].groupby(['uid','url']) \
      .agg({'facts': lambda x: x.tolist(),'fid': lambda x: set(x.tolist()), 'ts': lambda x: sorted_list(x)}) \
      .reset_index()
users_web_visits_df['number_of_visits'] = users_web_visits_df.fid.apply(lambda x: len(x))
users_web_visits_df.to_csv(f'{output_path}/user_website_visits.csv', index=False)